In [1]:
import pandas as pd
import numpy as np
import os

root_path = "save"
csv_dir = {}

for save_path in os.listdir(root_path):
    if save_path.__contains__("seed"):
        for fname in os.listdir(os.path.join(root_path, save_path, "0")):
            if fname.__contains__(".csv"):
                csv_name = os.path.join(root_path, save_path, "0", fname)
                str_seed_num = int(save_path.split("_")[-1])
                csv_dir[str_seed_num] = pd.read_csv(csv_name, header=2)

csv_dir

{7:            time  epoch  best_epoch  layer_to_update      lr   tr_loss  \
 0  1.713955e+09      0           0              110  0.0001  0.288984   
 1  1.713961e+09      1           1              110  0.0001  0.274005   
 2  1.713975e+09      2           1              110  0.0001  0.267762   
 3  1.713989e+09      3           1              110  0.0001  0.267953   
 
    val_loss     tr_rmse_a  tr_pcc_a_v  tr_pcc_a_conf  ... val_pcc_a_conf  \
 0  0.050542  [0.31228173]    0.452834            0.0  ...            0.0   
 1  0.049946  [0.26658487]    0.562892            0.0  ...            0.0   
 2  0.049882    [0.259237]    0.576217            0.0  ...            0.0   
 3  0.049582    [0.258914]    0.577102            0.0  ...            0.0   
 
               val_ccc_a     tr_rmse_v  tr_pcc_v_v tr_pcc_v_conf  \
 0  [0.5692512534230075]  [0.39655036]    0.416223           0.0   
 1  [0.5841120321081942]  [0.37423962]    0.471509           0.0   
 2  [0.5796150755013972]  [0.37146

In [2]:
from warnings import filterwarnings
filterwarnings('ignore')

csv_result_dict = {}

csv_dir = dict(sorted(csv_dir.items()))
for seed, result in csv_dir.items():
    csv_result_dict[seed] = {
        "tr_ccc_v": float(0),
        "tr_ccc_a": float(0),
        "val_ccc_v": float(0),
        "val_ccc_a": float(0)
    }
    
    result['tr_ccc_v'] = result['tr_ccc_v'].apply(lambda x:np.float(x[1:-1]))
    result['tr_ccc_a'] = result['tr_ccc_a'].apply(lambda x:np.float(x[1:-1]))
    result['val_ccc_v'] = result['val_ccc_v'].apply(lambda x:np.float(x[1:-1]))
    result['val_ccc_a'] = result['val_ccc_a'].apply(lambda x:np.float(x[1:-1]))
    
    csv_result_dict[seed]["tr_ccc_v"] = np.mean(result['tr_ccc_v'])
    csv_result_dict[seed]["tr_ccc_a"] = np.mean(result['tr_ccc_a'])
    csv_result_dict[seed]["val_ccc_v"] = np.mean(result['val_ccc_v'])
    csv_result_dict[seed]["val_ccc_a"] = np.mean(result['val_ccc_a'])
    
csv_result_dict

{0: {'tr_ccc_v': 0.4485386229053799,
  'tr_ccc_a': 0.5804362361287246,
  'val_ccc_v': 0.4051141701853094,
  'val_ccc_a': 0.6085749526062055},
 1: {'tr_ccc_v': 0.5119193588161877,
  'tr_ccc_a': 0.6132229647425829,
  'val_ccc_v': 0.45587322044725676,
  'val_ccc_a': 0.6113176851834222},
 2: {'tr_ccc_v': 0.48054415593143907,
  'tr_ccc_a': 0.585345728164845,
  'val_ccc_v': 0.4713909261705535,
  'val_ccc_a': 0.5774033253652706},
 3: {'tr_ccc_v': 0.48275491339402216,
  'tr_ccc_a': 0.6124773673992588,
  'val_ccc_v': 0.4264780983346771,
  'val_ccc_a': 0.5589069397801398},
 4: {'tr_ccc_v': 0.5117070359116404,
  'tr_ccc_a': 0.5977208479335414,
  'val_ccc_v': 0.44739732189490894,
  'val_ccc_a': 0.5118146865494567},
 5: {'tr_ccc_v': 0.4216548303025348,
  'tr_ccc_a': 0.5411939604294285,
  'val_ccc_v': 0.44742193208985687,
  'val_ccc_a': 0.5675169543865447},
 6: {'tr_ccc_v': 0.4118909524104157,
  'tr_ccc_a': 0.5812361931592677,
  'val_ccc_v': 0.3797732840872491,
  'val_ccc_a': 0.5954088060941488},
 7

In [3]:
# output_csv_name = "save/results/Seed0_12.csv"
output_csv_name = "save/results/Seed0_9.csv"

In [4]:
columns = [ "seed",
            "val_ccc_v",
            "val_ccc_a",
            "val_mean",
]

data = {c:[] for c in columns}

for s in csv_result_dict:
    if int(s) > 9: break
    data['seed'].append(s)
    data['val_ccc_a'].append(csv_result_dict[s]['val_ccc_a'])
    data['val_ccc_v'].append(csv_result_dict[s]['val_ccc_v'])
    
    val_mean = np.mean([csv_result_dict[s]['val_ccc_a'], csv_result_dict[s]['val_ccc_v']])
    data['val_mean'].append(val_mean)
    
print("std : ", f"{np.std(data['val_ccc_a']):.3f}", f"{np.std(data['val_ccc_v']):.3f}", f"{np.std(data['val_mean']):.3f}")
    
df = pd.DataFrame(data)
df.to_csv(output_csv_name, index=False)
display(df)
    
print("val_ccc_a : ",f"{np.mean(data['val_ccc_a']):.3f}")
print("val_ccc_v : ",f"{np.mean(data['val_ccc_v']):.3f}")
print("val_mean : ",f"{np.mean(data['val_mean']):.3f}")

res = []
res.append("result")
res.append(f"{np.mean(data['val_ccc_a']):.3f}")
res.append(f"{np.mean(data['val_ccc_v']):.3f}")
res.append(f"{np.mean(data['val_mean']):.3f}")

result_dict = {c:[r] for c, r in zip(columns,res)}

result_df = pd.DataFrame(result_dict)
result_df.to_csv(output_csv_name, mode='a', index=False, header=False)


std :  0.044 0.051 0.035


,seed,val_ccc_v,val_ccc_a,val_mean
0,0,0.405114,0.608575,0.506845
1,1,0.455873,0.611318,0.533595
2,2,0.471391,0.577403,0.524397
3,3,0.426478,0.558907,0.492693
4,4,0.447397,0.511815,0.479606
5,5,0.447422,0.567517,0.507469
6,6,0.379773,0.595409,0.487591
7,7,0.290825,0.583220,0.437023
8,8,0.368589,0.534988,0.451789
9,9,0.382420,0.465868,0.424144


val_ccc_a :  0.562
val_ccc_v :  0.408
val_mean :  0.485
